In [2]:
%load_ext autoreload
%autoreload 2
import os
import matplotlib.pyplot as plt
from os.path import join
import pandas as pd
import joblib
import sasc.config
import numpy as np
from collections import defaultdict
import pandas as pd
from sasc.modules.fmri_module import convert_module_num_to_voxel_num, add_stability_score
from sasc.config import FMRI_DIR, STORIES_DIR
import re
from math import ceil
from tqdm import tqdm
def extract_number(string):
    return int(re.findall(r'\d+', string)[0])

# Read all the info from stories into a single pickle file

In [3]:
# load stuff
# double check all of these, intro paragraph may be the same...
output_file = join(sasc.config.RESULTS_DIR,
                   'processed', "pilot6_story_data.pkl")
STORY_DATA_DIR = join(FMRI_DIR, 'brain_tune', 'story_data', '20241202')
story_mapping = {
    'roi/uts02___roi_nov30___seed=1': 'GenStory30.npy',
    'roi/uts02___roi_nov30___seed=2': 'GenStory31.npy',
    'roi/uts02___roi_nov30___seed=3': 'GenStory32.npy',
    'roi/uts02___roi_nov30___seed=7': 'GenStory33.npy',
    'roi/uts02___roi_nov30___seed=8': 'GenStory34.npy',
    # 'roi/uts02___roi_nov30___seed=9': 'GenStory35.npy',
}

# cluster_neighbors = joblib.load(join(FMRI_DIR, "voxel_neighbors_and_pcs", "cluster_neighbors_v1.pkl"))
perfs = joblib.load(join(sasc.config.FMRI_DIR, 'sasc', 'rj_models',
                    'opt_model', 'new_setup_performance.jbl'))

# add keys
stories_data_dict = defaultdict(list)
for story_idx, story_name in enumerate(story_mapping.keys()):
    # add scalar story descriptions
    stories_data_dict["story_name_original"].append(story_name)
    stories_data_dict["story_setting"].append(story_name.split("/")[0])
    stories_data_dict["story_name_new"].append(story_mapping[story_name])
    story_fname = [f for f in os.listdir(join(STORIES_DIR, story_name))
                   if f.startswith('uts02_story')][0]
    story_text = open(join(STORIES_DIR, story_name, story_fname), "r").read()
    stories_data_dict["story_text"].append(story_text)
    prompts_paragraphs = joblib.load(
        join(STORIES_DIR, story_name, "prompts_paragraphs.pkl")
    )
    prompts_paragraphs['paragraphs'] = story_text.split("\n\n")
    assert len(prompts_paragraphs['paragraphs']) == len(
        prompts_paragraphs['prompts'])

    # add paragraph-level descriptions
    timings = pd.read_csv(
        # join(STORIES_DIR, story_name, "timings_processed.csv")
        join(STORY_DATA_DIR,
             f'timings_processed{extract_number(story_mapping[story_name])}.csv'),
        header=None,
    ).rename(columns={0: 'word', 2: 'time_running'})
    timings['time_running'] *= 0.75
    # = timings['time_running'].apply(
    # lambda x: x * 0.75)
    # add offset to timings
    # timings['time_running']  # += 10  # -= 5  # += 6
    stories_data_dict["timing"].append(timings)
    stories_data_dict["prompts"].append(prompts_paragraphs["prompts"])
    stories_data_dict["paragraphs"].append(prompts_paragraphs["paragraphs"])

    # add paragraph-level metadata
    story_metadata_per_paragraph = pd.read_pickle(
        join(STORIES_DIR, story_name, "rows.pkl"))
    stories_data_dict["rows"].append(story_metadata_per_paragraph)

joblib.dump(stories_data_dict, output_file)

['/home/chansingh/automated-explanations/results/processed/pilot6_story_data.pkl']

In [4]:
stories_data_dict

defaultdict(list,
            {'story_name_original': ['roi/uts02___roi_nov30___seed=1',
              'roi/uts02___roi_nov30___seed=2',
              'roi/uts02___roi_nov30___seed=3',
              'roi/uts02___roi_nov30___seed=7',
              'roi/uts02___roi_nov30___seed=8'],
             'story_setting': ['roi', 'roi', 'roi', 'roi', 'roi'],
             'story_name_new': ['GenStory30.npy',
              'GenStory31.npy',
              'GenStory32.npy',
              'GenStory33.npy',
              'GenStory34.npy'],
             'story_text': ['As I stepped into the cavernous warehouse, the sheer scale of the operation overwhelmed me. The vats before me could hold several hundred gallons of liquid, and the ceiling soared above, creating an eight-thousand-square-foot expanse that seemed to stretch endlessly. The walls appeared as thick as two miles of solid concrete, insulating us from the outside world. Pallets of goods were stacked meticulously, each box labeled with precise mea

# resps

In [26]:
pilot_data_dir = join(sasc.config.PILOT_STORY_DATA_DIR, '20241202')

In [27]:
# load responses
resp_np_files = [stories_data_dict['story_name_new'][i].replace(
    '_resps', '') for i in range(len(stories_data_dict['story_name_new']))]
resps_dict = {
    k: np.load(join(pilot_data_dir, k))
    for k in tqdm(resp_np_files)
}

100%|██████████| 5/5 [00:05<00:00,  1.01s/it]


In [28]:
for i in range(len(stories_data_dict["story_name_new"])):
    story_name = stories_data_dict["story_name_new"][i]
    timings = stories_data_dict["timing"][i]
    # trs = ceil(timings['time_running'].max() * 0.75 / 2) - 10
    trs = ceil(timings['time_running'].max() / 2) - 10
    print(story_name, trs, resps_dict[story_name].shape[0])

    assert trs == resps_dict[story_name].shape[0]
    # print('resp trs', resps_dict[story_name].shape[0])
    # print(story_name.replace('_resps', ''))

GenStory30.npy 269 269
GenStory31.npy 224 224
GenStory32.npy 267 267
GenStory33.npy 210 210
GenStory34.npy 229 229
